In [1]:
import numpy as np
import mrcfile
from ccpi.io import *
from ccpi.framework import AcquisitionData, AcquisitionGeometry, ImageGeometry, ImageData
from ccpi.optimisation.algorithms import CGLS, PDHG, FISTA
from ccpi.optimisation.operators import BlockOperator, Gradient
from ccpi.optimisation.functions import L2NormSquared, ZeroFunction, MixedL21Norm, L1Norm

from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple
from ccpi.astra.processors import FBP

from ccpi.utilities.jupyter import islicer, link_islicer
from ccpi.utilities.display import plotter2D

# All external imports
import numpy as np
import os
import sys
import scipy

import numpy as np
import os
from ccpi.framework import AcquisitionData, AcquisitionGeometry, ImageData, ImageGeometry
import datetime
from PIL import Image

class TIFFWriter(object):
    
    def __init__(self,
                 **kwargs):
        
        self.data_container = kwargs.get('data_container', None)
        self.file_name = kwargs.get('file_name', None)
        
        if ((self.data_container is not None) and (self.file_name is not None)):
            self.set_up(data_container = self.data_container,
                        file_name = self.file_name)
        
    def set_up(self,
               data_container = None,
               file_name = None):
        
        self.data_container = data_container
        self.file_name = file_name
        
        if not ((isinstance(self.data_container, ImageData)) or 
                (isinstance(self.data_container, AcquisitionData))):
            raise Exception('Writer supports only following data types:\n' +
                            ' - ImageData\n - AcquisitionData')

    def write_file(self):
        '''alias of write'''
        return self.write()
    
    def write(self):
        ndim = len(self.data_container.shape)
        if ndim == 2:
            # save single slice
            with open(self.file_name, 'wb') as f:
                Image.fromarray(self.data_container.as_array()).save(f, 'tiff')
        elif ndim == 3:
            for sliceno in range(self.data_container.shape[0]):
                # save single slice
                # pattern = self.file_name.split('.')
                dimension = self.data_container.dimension_labels[0]
                fname = "{}_idx_{}.tiff".format(self.file_name.strip(".tiff"), sliceno)
                with open(fname, 'wb') as f:
                    Image.fromarray(self.data_container.as_array()[sliceno]).save(f, 'tiff')
        elif ndim == 4:
            for sliceno1 in range(self.data_container.shape[0]):
                # save single slice
                # pattern = self.file_name.split('.')
                dimension = [ self.data_container.dimension_labels[0] ]
                for sliceno2 in range(self.data_container.shape[1]):
                    idx = self.data_container.shape[0] * sliceno2 + sliceno1 
                    fname = "{}_{}_{}_idx_{}.tiff".format(self.file_name.strip(".tiff"), 
                        self.data_container.shape[0], self.data_container.shape[1], idx)
                    with open(fname, 'wb') as f:
                        Image.fromarray(self.data_container.as_array()[sliceno1][sliceno2]).save(f, 'tiff')
        else:
            raise ValueError('Cannot handle more than 4 dimensions')

In [3]:
# remove two angles for the aligned file
# angles should be in radians
angles = np.loadtxt('tomo_01.tlt') / 180. * np.pi

centre_slice = 2150

with mrcfile.open('tomo_01.ali') as mrc:
    #ndarray = mrc.data.copy()
    shape = mrc.data.shape
    # reconstruct on a smaller virtual panel
    vpanel_v_size = shape[2]#,3
    vox_size = mrc.voxel_size
    print (vox_size)
    # print (type(vox_size))
    vpanel_v_size = vpanel_v_size if vpanel_v_size != 1 else shape[2]
    
    ag = AcquisitionGeometry('parallel', '3D', 
                             pixel_num_h = shape[2],
                             pixel_num_v = shape[1],# vpanel_v_size, 
                             pixel_size_h = np.floor(np.float32(vox_size['y'])* 100 + 0.5 ) / 100.,
                             pixel_size_v = np.floor(np.float32(vox_size['z'])* 100 + 0.5 ) / 100.,
                             angles = angles[:-2],
                             angle_unit='radian',
                             dimension_labels=['angle', 'vertical', 'horizontal' ],
                            )
    tomo = ag.allocate(None)
    if vpanel_v_size == shape[2]:
        tomo.fill(mrc.data)
    else:
        band = int(vpanel_v_size/2)
        tomo.fill(mrc.data[:,:,centre_slice - band:centre_slice+band+1])
    # print (mrc.data.shape)
    
# "normalise" in min/max range. Could be better to do in 1-99 percentile.
tomo.subtract(tomo.min(), out=tomo)
tomo.divide(tomo.max()-tomo.min(), out=tomo)

(1.0999998, 1.0999995, 1.1)


In [4]:
if True:
    # remove borders and use contrast
    border_value = 1
    print(tomo.min(), tomo.max())
    for i,angle in enumerate(angles[1:-1]):
        dslice = tomo.subset(angle=i).as_array()
        
        # do it along horizontal axis
        x = int((shape[2]) / 2 * (1 - np.cos(angle)))
        print (x)
        if x != 0:
            dslice[:,0:x] = 1
            zero = np.ones_like(dslice) * border_value
            zero[:,x:-x] = dslice[:,x:-x]
        else:
            zero = dslice
        # dslice[:,:-x] = 0
        tomo.fill(zero, angle=i)
islicer (tomo, direction='angle')

0.0 1.0
877
795
715
638
565
495
430
369
312
259
211
168
129
95
66
42
23
10
2
0
2
10
23
41
64
92
125
163
206
253
305
362
423
488
557
629
706
785
873


interactive(children=(IntSlider(value=19, continuous_update=False, description='angle', max=38), FloatRangeSli…

IntSlider(value=19, continuous_update=False, description='angle', max=38)

In [5]:
# transpose the data to match what Astra projector expects
# take -log , remove zeros by adding epsilon
epsilon = 1e-7
tomost = -1 * (tomo.subset(dimensions=['vertical','angle','horizontal'])+epsilon).log()

In [ ]:
# islicer(tomost, direction='angle', cmap='viridis', size=10, minmax=(0,2))
# plotter2D(tomo.subset(angle=10), stretch_y=True)

In [6]:
# Create Image Geometry
# scale the output to test

scale = 5
vox_size = ag.pixel_size_h * scale

ig = ImageGeometry(voxel_num_x= int(ag.pixel_num_h/scale - (500./scale) ),
                   voxel_num_y= int(500 / scale), 
                   voxel_num_z= int(ag.pixel_num_v/scale - (500./scale)),
                   voxel_size_x=vox_size,
                   voxel_size_y=vox_size,
                   voxel_size_z=vox_size)


print (ig)
A = AstraProjector3DSimple(ig, ag)

Number of channels: 1
voxel_num : x667,y100,z642
voxel_size : x5.5,y5.5,z5.5
center : x0,y0,z0



In [ ]:
# x_init = ig.allocate(0) 

In [7]:
from ccpi.optimisation.operators import Identity
from ccpi.framework import BlockDataContainer
# L = Identity(ig)
L = Gradient(ig)
if True:
    if scale == 5:
        alpha = 111.
    elif scale == 1:
        alpha = 49.96
    elif scale == 2:
        alpha = 72
else:
    # alpha prop to ratio of norm of A and norm of L
    print ("calculating norm of A")
    normA = A.norm()
    print ("calculating norm of Gradient")
    normL = L.norm()
    ratio = normA/normL

    # gamma selects the weighing between the regularisation and the fitting term:
    # 1 means equal weight
    # 
    gamma = 1 
    alpha = gamma * ratio
    print (alpha, normA, normL, normL/normA, normA/normL)


Initialised GradientOperator with C backend running with  16  threads


In [ ]:
class LCGLS(CGLS):

    def set_up(self, x_init, operator, data, tolerance=1e-6):
        '''initialisation of the algorithm

        :param operator: Linear operator for the inverse problem
        :param x_init: Initial guess ( Default x_init = 0)
        :param data: Acquired data to reconstruct       
        :param tolerance: Tolerance/ Stopping Criterion to end CGLS algorithm
        '''
        print("{} setting up".format(self.__class__.__name__, ))
        
        self.x = x_init * 0.
        self.operator = operator
        self.tolerance = tolerance

        self.r = data - self.operator.direct(self.x)
        
        # self.s = self.operator.adjoint(self.r)
        # self.p = self.s.copy()
        self.p = self.operator.adjoint(self.r)
        self.s = self.p.copy()

        self.q = self.operator.range_geometry().allocate()
        self.norms = self.p.norm()
        self.norms0 = self.norms
        self.gamma = self.norms**2
        self.normx = self.x.norm()
        # self.xmax = self.normx   
        
        self.loss.append(self.r.squared_norm())
        self.configured = True
        print("{} configured".format(self.__class__.__name__, ))
     
    def update(self):
        '''single iteration'''
        
        self.operator.direct(self.p, out=self.q)
        delta = self.q.squared_norm()
        # print ("delta {} self.q.shape {}".format(delta, self.q.shape))
        alpha = self.gamma/delta
        # update residuals
        self.r.axpby(1, -alpha, self.q, out=self.r)
        
        # update solution
        self.x.axpby(1, alpha, self.p, out=self.x)
        #self.x += alpha * self.p
        #self.r -= alpha * self.q
        
        self.operator.adjoint(self.r, self.s)
        
        self.norms = self.s.norm()
        # print ("self.norms {}".format(self.norms))
        self.gamma1 = self.gamma
        self.gamma = self.norms**2
        self.beta = self.gamma/self.gamma1
        #self.p = self.s + self.beta * self.p   
        self.p.axpby(self.beta, 1, self.s, out=self.p)
        

In [ ]:
operator_block = BlockOperator( A, alpha * L)

zero_data = L.range_geometry().allocate(0)

data_block = BlockDataContainer(tomost, zero_data)
  
cgls_regularised = LCGLS(operator=operator_block, data=data_block, 
                        update_objective_interval = 10)
cgls_regularised.max_iteration = 10000

cgls_regularised.run(20)


In [8]:
from ccpi.optimisation.algorithms import CGLS
import numpy

import multiprocessing as mp
from contextlib import closing

import time

def save_callback(iteration, obj, solution):
    writer = NEXUSDataWriter()
    writer.set_up(data_container=solution, 
                  file_name="./scale_2_gamma1_it_{}.nxs".format(iteration)
    )
    writer.write_file()

class LRCGLS(CGLS):
    '''Regularised CGLS

    Tikhonov regularisation
    '''
    def __init__(self, x_init=None, operator=None, data=None, tolerance=1e-6, **kwargs):
        '''initialisation of the algorithm

        :param operator : Linear operator for the inverse problem
        :param x_init : Initial guess ( Default x_init = 0)
        :param data : Acquired data to reconstruct       
        :param tolerance: Tolerance/ Stopping Criterion to end CGLS algorithm
        '''
        super(CGLS, self).__init__(**kwargs)
        
        alpha = kwargs.get('alpha', None)
        preallocate = kwargs.get('preallocate', False)

        if alpha is None:
            raise ValueError('Please specify alpha')

        if x_init is None and operator is not None:
            x_init = operator.domain_geometry().allocate(0)
        if x_init is not None and operator is not None and data is not None:
            if preallocate:
                self.set_up(x_init=x_init, operator=operator, data=data, tolerance=tolerance, alpha=alpha)
            else:
                self.set_up_nopreallocate(x_init, operator, data, tolerance, alpha)
                self.update_back = self.update
                self.update = self.update_nopreallocate

    def set_up_nopreallocate(self, x_init, operator, data, tolerance=1e-6, alpha=1e-2):
        '''initialise the algorithm with minimal memory footprint'''
        print("{} setting up".format(self.__class__.__name__, ))
        
        self.x = x_init * 0.
        data = BlockDataContainer(data, 0)
        gradient = Gradient(operator.domain_geometry(), backend='c')
        gradient.scalar = alpha
        self.operator = BlockOperator(operator, gradient)
        self.tolerance = tolerance

        # self.r = data - self.operator.direct(self.x)
        self.r = BlockDataContainer(- operator.direct(self.x), -gradient.scalar * gradient.direct(self.x)) + data
        #self.s = self.operator.adjoint(self.r)
        self.s = gradient.adjoint(self.r.get_item(1))
        self.s.multiply(gradient.scalar, out=self.s)
        self.s.add(operator.adjoint(self.r.get_item(0)), out=self.s)

        self.p = self.s.copy()
        # don't need to preallocate as we reallocate at each iteration
        # self.q = self.operator.range_geometry().allocate()
        self.norms0 = self.s.norm()
        
        self.norms = self.s.norm()

        self.gamma = self.norms0**2
        self.normx = self.x.norm()
        self.xmax = self.normx   
        
        self.loss.append(self.r.squared_norm())
        self.configured = True
        print("{} configured".format(self.__class__.__name__, ))
    
    def update_nopreallocate(self):
        '''single iteration'''
        gradient = self.operator.get_item(0,1)

        term0 = self.operator.get_item(0,0).direct(self.p)
        delta0 = term0.squared_norm()
        # self.operator.get_item(0,0).direct(self.p, out=self.q.get_item(0))
        # delta0 = self.q.get_item(0).squared_norm()
        delta1, term1 = self.operator.get_item(0,1).direct_L21norm(self.p)
        #term1.multiply(gradient.scalar, out=self.q.get_item(1))
        term1.multiply(gradient.scalar, out=term1)
        q = BlockDataContainer(term0, term1)

        delta = delta0 + delta1
        # print ("delta", delta, "delta0", delta0, "delta1", delta1)
        # delta = self.q.squared_norm()

        alpha = self.gamma/delta
        # print ("alpha", alpha)                        
        # self.x += alpha * self.p
        # self.r -= alpha * self.q
        
        self.x.axpby(1, alpha, self.p, out=self.x)
        #self.x += alpha * self.p
        self.r.axpby(1, -alpha, q, out=self.r)
        del q, term0, term1
        #self.r -= alpha * self.q

        # adjoint of block operator is sum of adjoints
        # sumsq, self.s = self.operator.adjoint(self.r)
        term0 = self.operator.get_item(0,0).adjoint(self.r.get_item(0))
        delta0 = term0.norm()
        delta1, term1 = gradient.adjoint_L21norm(self.r.get_item(1))
        self.norms = delta0 + numpy.sqrt(delta1)
        # self.norms = self.s.norm()

        #term0.add(term1, out=self.s)
        term0.axpby(1,gradient.scalar, term1, out=self.s)
        # print ("self.norms {}".format(self.norms))
        
        self.gamma1 = self.gamma
        self.gamma = self.norms**2
        self.beta = self.gamma/self.gamma1
        self.p.axpby(self.beta, 1., self.s , out=self.p)
        
        self.normx = self.x.norm()
        self.xmax = numpy.maximum(self.xmax, self.normx)


    def set_up(self, x_init, operator, data, tolerance=1e-6, alpha=1e-2):
        '''initialisation of the algorithm

        :param operator: Linear operator for the inverse problem
        :param x_init: Initial guess ( Default x_init = 0)
        :param data: Acquired data to reconstruct       
        :param tolerance: Tolerance/ Stopping Criterion to end CGLS algorithm
        :param alpha: regularisation parameter
        '''
        print("{} setting up".format(self.__class__.__name__, ))
        
        self.x = x_init * 0.
        #data = BlockDataContainer(data, operator.domain_geometry().allocate(0))
        data = BlockDataContainer(data, 0)
        gradient = Gradient(operator.domain_geometry(), backend='c')
        gradient.scalar = alpha
        self.operator = BlockOperator(operator, gradient)
        self.tolerance = tolerance
        self.q = self.operator.range_geometry().allocate()

        # self.r = data - self.operator.direct(self.x)
        # self.r = data - BlockDataContainer(operator.direct(self.x), gradient.scalar * gradient.direct(self.x))
        self.r = BlockDataContainer(- operator.direct(self.x), - gradient.scalar * gradient.direct(self.x)) + data
        #self.s = self.operator.adjoint(self.r)
        self.p = gradient.adjoint(self.r.get_item(1))
        self.p.multiply(gradient.scalar, out=self.p)
        self.p.add(operator.adjoint(self.r.get_item(0)), out=self.p)
        self.s = self.p.copy()
        self.s1 = gradient.domain_geometry().allocate(None)

        self.norms = self.p.norm()
        self.norms0 = self.norms
        self.gamma = self.norms**2
        self.normx = self.x.norm()
        
        
        self.loss.append(self.r.squared_norm())
        self.configured = True
        print("{} configured".format(self.__class__.__name__, ))
     
    def update(self):
        '''single iteration'''
        operator = self.operator.get_item(0,0)
        gradient = self.operator.get_item(0,1)
        operator.direct(self.p, out=self.q.get_item(0))
        delta0 = self.q.get_item(0).squared_norm()
        # self.operator.get_item(0,0).direct(self.p, out=self.q.get_item(0))
        # delta0 = self.q.get_item(0).squared_norm()
        delta1, a = gradient.direct_L21norm(self.p, out=self.q.get_item(1))
        #term1.multiply(gradient.scalar, out=self.q.get_item(1))
        self.q.get_item(1).multiply(gradient.scalar, out=self.q.get_item(1))
        
        delta = delta0 + delta1 * numpy.abs(gradient.scalar)
        # print ("delta", delta, "delta0", delta0, "delta1", delta1)
        # delta = self.q.squared_norm()

        alpha = self.gamma/delta
        t0 = time.time()
        self.r.axpby(1, -alpha, self.q, out=self.r)
        
        # print ("alpha", alpha)                        
        # self.x += alpha * self.p
        # self.r -= alpha * self.q
        
        self.x.axpby(1, alpha, self.p, out=self.x)
        #self.x += alpha * self.p
        #self.r -= alpha * self.q

        # adjoint of block operator is sum of adjoints
        # sumsq, self.s = self.operator.adjoint(self.r)
        
        operator.adjoint(self.r.get_item(0), out=self.s)
        delta0 = self.s.norm()

        delta1, a = gradient.adjoint_L21norm(self.r.get_item(1), out=self.s1)
        self.norms = delta0 + numpy.sqrt(delta1) * gradient.scalar
        # self.norms = self.s.norm()

        #term0.add(term1, out=self.s)
        self.s.axpby(1,gradient.scalar, self.s1, out=self.s)
        
        # print ("self.norms {}".format(self.norms))
        
        self.gamma1 = self.gamma
        self.gamma = self.norms**2
        self.beta = self.gamma/self.gamma1
        self.p.axpby(self.beta, 1., self.s , out=self.p)
        
        self.normx = self.x.norm()
        # self.xmax = numpy.maximum(self.xmax, self.normx)



from ccpi.optimisation.algorithms import RCGLS
rcgls = LRCGLS(operator=A, data = tomost, 
              update_objective_interval = 10, 
             max_iteration=100, alpha=alpha, 
             preallocate=True)


# rcgls.operator.get_item(0,1).adjoint_L21norm = rcgls.operator.get_item(0,1).operator.adjoint_L21norm
rcgls.run(50, callback=save_callback)


LRCGLS setting up
Initialised GradientOperator with C backend running with  16  threads
LRCGLS configured
     Iter   Max Iter     Time/Iter            Objective
                               [s]                     
        0        100         0.000          4.33306e+08
       10        100        15.082          4.09039e+08
       20        100        11.270          3.62040e+08
       30        100        13.996          3.48920e+08
       40        100        11.957          3.41276e+08
       50        100        14.702          3.35920e+08
       50        100        14.702          3.35920e+08
Stop criterion has been reached.


In [ ]:

a = (cgls_regularised.get_output()/rcgls.get_output())
print (a.min(), a.max())
print (cgls_regularised.get_output().min(), cgls_regularised.get_output().max())
print (rcgls.get_output().min(), rcgls.get_output().max())


In [ ]:
print (type (cgls_regularised.get_output()), type(rcgls.get_output()))
from ccpi.utilities.jupyter import link_islicer
sl1 = islicer(rcgls.get_output(), direction='horizontal_y')
sl2 = islicer(cgls_regularised.get_output(), direction='horizontal_y')
sl3 = islicer((cgls_regularised.get_output()/rcgls.get_output()), direction='horizontal_y', minmax=(0.9988101, 0.9988458))
link_islicer(sl1,sl2, sl3)

In [ ]:
plotter2D(cgls_regularised.get_output().subset(horizontal_y=2000), stretch_y=True)

In [ ]:
islicer(cgls_regularised.get_output(), 
        direction='horizontal_y', 
        slice_number = 50,
        cmap='viridis', 
        size=12, 
        minmax=(2.e-4, 3.e-4))

In [9]:
# cgls_regularised.run(20)
islicer(rcgls.get_output(), 
        direction='horizontal_y', 
        slice_number = 64 ,
        cmap='viridis', 
        size=12)

interactive(children=(IntSlider(value=64, continuous_update=False, description='horizontal_y', max=99), FloatR…

IntSlider(value=64, continuous_update=False, description='horizontal_y', max=99)

In [ ]:
islicer(cgls_regularised.get_output(),  direction='horizontal_x', cmap='viridis', size=12, minmax=(0.4e-5,6.5e-6))

In [ ]:
import ccpi


from ccpi.io import NEXUSDataWriter
writer = NEXUSDataWriter()
writer.set_up(data_container=cgls_regularised.get_output(), file_name="./protein.nxs")
writer.write_file()

In [ ]:
from ccpi.optimisation.operators import BlockOperator
from ccpi.optimisation.functions import BlockFunction, IndicatorBox

# Define Gradient Operator and BlockOperator 
Grad = Gradient(ig)
K = BlockOperator(Grad,A)

# Define BlockFunction F using the MixedL21Norm() and the L2NormSquared()
alpha = 20.
f1 = alpha * MixedL21Norm()
f2 = L2NormSquared(b=tomost)
F = BlockFunction(f1,f2)

# Define BlockFunction

In [ ]:
# G, as a positivity constraint
G = IndicatorBox(lower=0)

# Compute operator norm and choose step-size sigma and tau such that sigma*tau||K||^{2}<1
normK =  K.norm()
sigma = 1
tau = 1/(sigma*normK**2)

pdhg = PDHG(f = F, g = G, operator = K, tau = tau, sigma = sigma, 
            max_iteration = 1000, update_objective_interval = 1)

pdhg.update_objective_interval = 10
pdhg.run(90, very_verbose=True)

algo = PDHG(max_iteration=100, operator=A, f=L2NormSquared(b=tomost), g=ZeroFunction(), 
            tau=tau, sigma=sigma)
algo.run(5, very_verbose=True)

In [ ]:
islicer(pdhg.get_output(), direction='vertical', cmap='viridis', size=10)

In [ ]:
with mrcfile.open('tomo_01.ali') as mrc:
    #ndarray = mrc.data.copy()
    shape = mrc.data.shape
    vox_size = np.asarray(mrc.voxel_size)
    print (vox_size)


In [ ]:
from ccpi.optimisation.functions import LeastSquares
f = LeastSquares(A, tomost, c=0.5)
print (f(cgls_regularised.get_output()))

In [ ]:
print (cgls_regularised.r.squared_norm()/tomost.size)
print (tomost.size)

In [ ]:
import numpy as np
import os
from ccpi.framework import AcquisitionData, AcquisitionGeometry, ImageData, ImageGeometry
import datetime
from PIL import Image

class TIFFWriter(object):
    
    def __init__(self,
                 **kwargs):
        
        self.data_container = kwargs.get('data_container', None)
        self.file_name = kwargs.get('file_name', None)
        
        if ((self.data_container is not None) and (self.file_name is not None)):
            self.set_up(data_container = self.data_container,
                        file_name = self.file_name)
        
    def set_up(self,
               data_container = None,
               file_name = None):
        
        self.data_container = data_container
        self.file_name = file_name
        
        if not ((isinstance(self.data_container, ImageData)) or 
                (isinstance(self.data_container, AcquisitionData))):
            raise Exception('Writer supports only following data types:\n' +
                            ' - ImageData\n - AcquisitionData')

    def write_file(self):
        '''alias of write'''
        return self.write()
    
    def write(self):
        ndim = len(self.data_container.shape)
        if ndim == 2:
            # save single slice
            with open(self.file_name, 'wb') as f:
                Image.fromarray(self.data_container.as_array()).save(f, 'tiff')
        elif ndim == 3:
            for sliceno in range(self.data_container.shape[0]):
                # save single slice
                # pattern = self.file_name.split('.')
                dimension = self.data_container.dimension_labels[0]
                fname = "{}_idx_{}.tiff".format(self.file_name.strip(".tiff"), sliceno)
                with open(fname, 'wb') as f:
                    Image.fromarray(self.data_container.as_array()[sliceno]).save(f, 'tiff')
        elif ndim == 4:
            for sliceno1 in range(self.data_container.shape[0]):
                # save single slice
                # pattern = self.file_name.split('.')
                dimension = [ self.data_container.dimension_labels[0] ]
                for sliceno2 in range(self.data_container.shape[1]):
                    idx = self.data_container.shape[0] * sliceno2 + sliceno1 
                    fname = "{}_{}_{}_idx_{}.tiff".format(self.file_name.strip(".tiff"), 
                        self.data_container.shape[0], self.data_container.shape[1], idx)
                    with open(fname, 'wb') as f:
                        Image.fromarray(self.data_container.as_array()[sliceno1][sliceno2]).save(f, 'tiff')
        else:
            raise ValueError('Cannot handle more than 4 dimensions')

In [ ]:
print(cgls_regularised.get_output().shape)

writer = TIFFWriter()
writer.set_up(data_container=cgls_regularised.get_output(), file_name="cgls_regularised")
writer.write()

In [ ]:
print(np.linspace(0,np.pi, 10))
# AcquisitionGeometry??
ag = AcquisitionGeometry(geom_type='cone', pixel_num_h=10, pixel_num_v=11, channels=2, angles = np.linspace(0,np.pi, 9))
ad = ag.allocate(value='random')

print (ad.dtype, ad.shape)
import re
# re.match??
match = re.search('.tiff', "test4d.tiff")
print (match)
print ("match", re.search('.tiff', "test4d.tiff"))
writer = TIFFWriter()
writer.set_up(data_container=ad, file_name="test4d.tiff")
writer.write()
# !rm *.tiff

In [ ]:
!ls
dslice = np.array(Image.open('cgls_regularised_idx_1.tiff'))
plotter2D(dslice, cmap='viridis')

In [ ]:
!ls
for i in range(10):
    print ("a_{:04d}".format(i))

In [ ]:
ig = ImageGeometry(1,2,3)
a = ig.allocate(1)
b = ig.allocate(2)
c = ig.allocate(3)
bdc0 = BlockDataContainer(a,b)

bdc1 = BlockDataContainer(c,0)

bdc0 - bdc1